In [27]:
import pandas as pd
import os
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Basisverzeichnisse
input_dir = r"E:\MA_data\raw production history ENTSO-E"
output_dir = r"C:\Users\alexa\Documents\Webapp\data\production_history\processed_new"

# Liste der Monate von 2015-01 bis 2024-10 generieren
months = pd.date_range(start="2015-02", end="2015-03", freq="MS").strftime("%Y_%m").tolist()

# For-Schleife für jede Datei
for month in months:
    # Dateipfad erstellen
    input_file = os.path.join(input_dir, f"{month}_ActualGenerationOutputPerGenerationUnit_16.1.A_r2.1.csv")
    output_file = os.path.join(output_dir, f"production_summary_{month}.xlsx")

    # Überprüfen, ob die Datei existiert
    if not os.path.exists(input_file):
        print(f"Datei nicht gefunden: {input_file}")
        continue

    # Datei einlesen
    print(f"Bearbeite Datei: {input_file}")
    data = pd.read_csv(input_file, sep='\t')

    # Filtere nach GenerationUnitType == 'Wind Onshore' oder 'Wind Offshore'
    filtered_data = data[(data['GenerationUnitType'] == 'Wind Onshore ') | (data['GenerationUnitType'] == 'Wind Offshore ')]

    # Konvertiere 'DateTime (UTC)' in datetime
    filtered_data.loc[:, 'DateTime (UTC)'] = pd.to_datetime(filtered_data['DateTime (UTC)'])

    # Wichtige Spalten identifizieren, 'AreaCode', 'AreaDisplayName', 'AreaTypeCode' and 'MapCode' of identical WPPs may differ --> use at least one of them as a criterion to identify unique windfarms, and sort out the duplicates manually, because otherwise, the production data are appended twice to the same wind farm
    unique_windfarms = filtered_data[['GenerationUnitName', 'GenerationUnitCode', 'GenerationUnitType', 'GenerationUnitInstalledCapacity(MW)', 'AreaCode']].drop_duplicates()

    # Listen für die Produktion zu jeder Stunde hinzufügen
    production_data = []
    for _, row in unique_windfarms.iterrows():
        # Filtern der Daten für das aktuelle Windkraftwerk
        windfarm_data = filtered_data[
            (filtered_data['GenerationUnitName'] == row['GenerationUnitName']) &
            (filtered_data['AreaCode'] == row['AreaCode']) # important to avoid adding to a wind farm production data of all its duplicates
        ]

        # Erstelle Liste mit Sublisten mit Zeit und Produktion. 2D-Array nicht sinnvoll, da keine Kommas eingefügt werden und Zeilenumbrüche zwischen den Sublisten entstehen --> Schwiereigkeiten beim erneuten Einlesen
        production_array = [
            [time, production]
            for time, production in zip(
                windfarm_data['DateTime (UTC)'],
                windfarm_data['ActualGenerationOutput(MW)']
            )
            if pd.notna(production) and time.minute == 0  # Nur volle Stunden übernehmen (Resolution of weather data is hourly) und fehlende Werte überspringen
        ]

        # Daten für das Windkraftwerk hinzufügen
        row_data = {
            'GenerationUnitName': row['GenerationUnitName'],
            'GenerationUnitCode': row['GenerationUnitCode'],
            'GenerationUnitType': row['GenerationUnitType'],
            'GenerationUnitInstalledCapacity(MW)': row['GenerationUnitInstalledCapacity(MW)'],
            'Production': production_array
        }
        production_data.append(row_data)

    # DataFrame erstellen und in Excel speichern
    output_df = pd.DataFrame(production_data)
    output_df.to_excel(output_file, index=False)

    print(f"Excel-Datei wurde erfolgreich erstellt: {output_file}")

Bearbeite Datei: E:\MA_data\raw production history ENTSO-E\2015_02_ActualGenerationOutputPerGenerationUnit_16.1.A_r2.1.csv
Excel-Datei wurde erfolgreich erstellt: C:\Users\alexa\Documents\Webapp\data\production_history\processed_new\production_summary_2015_02.xlsx
Bearbeite Datei: E:\MA_data\raw production history ENTSO-E\2015_03_ActualGenerationOutputPerGenerationUnit_16.1.A_r2.1.csv
Excel-Datei wurde erfolgreich erstellt: C:\Users\alexa\Documents\Webapp\data\production_history\processed_new\production_summary_2015_03.xlsx


In [29]:
import pandas as pd
import os
import ast

# Verzeichnisse
input_dir = r"C:\Users\alexa\Documents\Webapp\data\production_history\processed_new"
output_file = r"C:\Users\alexa\Documents\Webapp\data\production_history\production_summary_all.xlsx"

# Liste der Monate
months = pd.date_range(start="2015-01", end="2015-03", freq="MS").strftime("%Y_%m").tolist()
columns_merge = ['GenerationUnitName', 'GenerationUnitCode', 'GenerationUnitType', 'GenerationUnitInstalledCapacity(MW)']
final_df = pd.DataFrame(columns=columns_merge + ['Production'])

# Sicherstellen, dass Production korrekt geparst wird
def safe_eval_production(value):
    time = []
    production = []
    for _, element in enumerate(value):
        
        element = element.replace("[", "", 1)
        element = element[:-1]
        element = element.replace("Timestamp(", "").replace(")", "")
        
        element = ast.literal_eval(f"[{element}]")
        
        time.append([item[0] for item in element])
        production.append([item[1] for item in element])

    return time, production

# Verarbeitung der Dateien
for month in months:
    input_file = os.path.join(input_dir, f"production_summary_{month}.xlsx")
    if not os.path.exists(input_file):
        print(f"Datei nicht gefunden: {input_file}")
        continue

    print(f"Verarbeite Datei: {input_file}")
    df = pd.read_excel(input_file)
    
    # Processing der Production-Spalte
    time, production = safe_eval_production(df['Production'])

    # Zusammenführen: Gleiche Windkraftanlagen zusammenführen
    if final_df.empty:
        final_df = df
    else:
        final_df = final_df.merge(df, on=columns_merge, how='outer', suffixes=('', '_new'))
        final_df['Production'] = final_df.apply(
            lambda row: (row['Production'] if isinstance(row['Production'], list) else []) +
                        (row['Production_new'] if isinstance(row.get('Production_new', []), list) else []),
            axis=1
        )
        final_df.drop(columns=['Production_new'], inplace=True)

# Ergebnis speichern
final_df.to_excel(output_file, index=False)
print(f"Zusammengeführte Excel-Tabelle wurde gespeichert unter: {output_file}")


Verarbeite Datei: C:\Users\alexa\Documents\Webapp\data\production_history\processed_new\production_summary_2015_01.xlsx
Verarbeite Datei: C:\Users\alexa\Documents\Webapp\data\production_history\processed_new\production_summary_2015_02.xlsx
Verarbeite Datei: C:\Users\alexa\Documents\Webapp\data\production_history\processed_new\production_summary_2015_03.xlsx
Zusammengeführte Excel-Tabelle wurde gespeichert unter: C:\Users\alexa\Documents\Webapp\data\production_history\production_summary_all.xlsx


In [10]:
import ast

a = "[['2015-01-01 00:00:00', 39.73], ['2015-01-01 00:30:00', 35.32]]"
print(a, type(a))
a = a.replace("[", "", 1)
a = a[:-1]
print(a)

a = ast.literal_eval(f"[{a}]")
print(a, type(a))
# Daten extrahieren
dates = [item[0] for item in a]

print(dates)
values = [item[1] for item in a]

[['2015-01-01 00:00:00', 39.73], ['2015-01-01 00:30:00', 35.32]] <class 'str'>
['2015-01-01 00:00:00', 39.73], ['2015-01-01 00:30:00', 35.32]
[['2015-01-01 00:00:00', 39.73], ['2015-01-01 00:30:00', 35.32]] <class 'list'>
['2015-01-01 00:00:00', '2015-01-01 00:30:00']
